In [1]:
import pandas as pd
import requests
import json
import logging
import random
import time

# --- 1. Logging Setup ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- 2. LLM Configuration ---
OLLAMA_BASE_URL = "http://localhost:11434"

# Define the list of Ollama models to test for SRS generation
OLLAMA_MODELS_TO_TEST = [
    "llama2",
    "mistral",
    "llama3:8b",
    "gemma:7b",
    "phi3:mini"
]

# --- 3. Data Loading and Preparation (Global Fixed Sample) ---
print("--- Initializing Data Loading and Fixed Sample Preparation ---")

# Load NFR Data
nfr_file_path = "datasets/NFR.xlsx" # Ensure this path is correct for your NFR data
nfr_sheets = pd.read_excel(nfr_file_path, sheet_name=None)
nfr_data_raw = pd.concat(nfr_sheets.values(), ignore_index=True)
nfr_data = nfr_data_raw[['User Review Sentence', 'NFR class']].rename(
    columns={'User Review Sentence': 'review', 'NFR class': 'ground_truth'}
)
nfr_data['ground_truth'] = nfr_data['ground_truth'].str.strip().str.lower()
VALID_NFR_LABELS = ["usability", "reliability", "performance", "portability", "security", "other"]
nfr_data = nfr_data[nfr_data['ground_truth'].isin(VALID_NFR_LABELS)].reset_index(drop=True)

# Sample NFR reviews
sampled_nfr_reviews_fixed = nfr_data.to_dict('records')

# Format the combined classified feedback list (to be shuffled consistently later)
global_classified_feedback_list = []
for item in sampled_nfr_reviews_fixed:
    global_classified_feedback_list.append(f"- {item['ground_truth'].title()}: {item['review']}")

# Shuffle this list ONCE with the fixed seed to get the master input order
random.shuffle(global_classified_feedback_list)

# The final classified feedback string used for ALL experiments
CLASSIFIED_FEEDBACK_INPUT_GLOBAL = "\n".join(global_classified_feedback_list)

print(f"Global Fixed Sample Prepared: {len(global_classified_feedback_list)} reviews (NFRs: {len(sampled_nfr_reviews_fixed)}).")
print("This exact sample will be used for all prompt strategies.")
print("-" * 40)

# You can now print this variable directly in a cell to see the sample:
# print(CLASSIFIED_FEEDBACK_INPUT_GLOBAL)


# --- 4. Define All SRS Generation Prompts ---
# Base Structured Instructional Prompt (as identified earlier)
BASE_PROMPT = """
Based on the following classified user feedback, generate a comprehensive Software Requirements Specification (SRS) document.

Classified Feedback:
{classified_feedback}

Generate a well-structured SRS document that includes the following sections, using clear and concise language:
1. Introduction and Purpose
2. System Overview
3. Functional Requirements
4. Non-Functional Requirements
5. System Features
6. External Interfaces
7. Performance Requirements
8. Design Constraints

Ensure the generated SRS is coherent, logically organized, and directly addresses the classified feedback provided.
"""

# Prompt 1: Role-Based Prompting
ROLE_BASED_PROMPT = """
As a **Senior Software Requirements Engineer** with over 10 years of experience in drafting comprehensive and high-quality Software Requirements Specification (SRS) documents, analyze the following classified user feedback. Your expertise should ensure the SRS is professional, precise, and highly actionable for a development team.

Classified Feedback:
{classified_feedback}

Generate a well-structured SRS document that includes the following sections, using clear, concise, and unambiguous language:
1. Introduction and Purpose
2. System Overview
3. Functional Requirements
4. Non-Functional Requirements
5. System Features
6. External Interfaces
7. Performance Requirements
8. Design Constraints

Ensure the generated SRS is coherent, logically organized, and directly addresses the classified feedback provided, focusing on **what** the system shall do, rather than **how** it is implemented.
"""

# Prompt 2: Constraint-Based Prompting
CONSTRAINT_BASED_PROMPT = """
Based on the following classified user feedback, generate a comprehensive Software Requirements Specification (SRS) document.

Classified Feedback:
{classified_feedback}

Generate a well-structured SRS document that includes the following sections, using clear and concise language:
1. Introduction and Purpose
2. System Overview
3. Functional Requirements
4. Non-Functional Requirements
5. System Features
6. External Interfaces
7. Performance Requirements
8. Design Constraints

Crucially, ensure the generated SRS is coherent, logically organized, and directly addresses the classified feedback provided. **STRICTLY avoid including any implementation details, specific design solutions, or technical jargon that is not directly inferable from the user feedback. Each requirement must be unique and non-redundant. Ensure all requirements are stated in a clear, testable, and unambiguous manner.**
"""

# Prompt 3: Few-Shot Prompting
FEW_SHOT_PROMPT = """
Based on the following classified user feedback, generate a comprehensive Software Requirements Specification (SRS) document.

Here are examples of how classified user feedback should be effectively translated into a precise and actionable SRS requirement:
- **Example Feedback:** "The app crashes frequently."
- **Example SRS Requirement:** "The system shall exhibit a crash-free operation rate of 99.9% during normal usage, ensuring application stability."

- **Example Feedback:** "I want to send high-res photos."
- **Example SRS Requirement:** "The system shall support the transmission of images up to 10MB in resolution while maintaining optimal performance."

Classified Feedback:
{classified_feedback}

Generate a well-structured SRS document that includes the following sections, using clear and concise language:
1. Introduction and Purpose
2. System Overview
3. Functional Requirements
4. Non-Functional Requirements
5. System Features
6. External Interfaces
7. Performance Requirements
8. Design Constraints

Ensure the generated SRS is coherent, logically organized, and directly addresses the classified feedback provided.
"""

# Prompt 4: Chain-of-Thought (CoT) Inspired Prompting
COT_PROMPT = """
Carefully and methodically analyze each piece of the following classified user feedback. First, identify the core user need or issue for each feedback item. Then, systematically synthesize these into a comprehensive Software Requirements Specification (SRS) document. Think step-by-step about how each piece of feedback translates into a formal requirement.

Classified Feedback:
{classified_feedback}

Generate a well-structured SRS document that includes the following sections, using clear and concise language:
1. Introduction and Purpose
2. System Overview
3. Functional Requirements
4. Non-Functional Requirements
5. System Features
6. External Interfaces
7. Performance Requirements
8. Design Constraints

Ensure the generated SRS is coherent, logically organized, and directly addresses the classified feedback provided.
"""

# --- 5. LLM Interaction Function ---
def generate_srs_with_ollama(classified_feedback_text: str, model_name: str, prompt_text: str) -> dict:
    """
    Sends a request to the local Ollama model to generate an SRS document.
    """
    url = f"{OLLAMA_BASE_URL}/api/generate"
    headers = {"Content-Type": "application/json"}

    formatted_prompt = prompt_text.format(classified_feedback=classified_feedback_text)

    data = {
        "model": model_name,
        "prompt": formatted_prompt,
        "stream": False,
        "options": {
            "temperature": 0.5,
            "num_predict": 4096
        }
    }

    try:
        response = requests.post(url, headers=headers, data=json.dumps(data), timeout=600)
        response.raise_for_status()
        result = response.json()
        return {"success": True, "raw_response": result.get("response", "")}
    except requests.exceptions.ConnectionError:
        logger.error(f"Failed to connect to Ollama server at {OLLAMA_BASE_URL}. Is Ollama running?")
        return {"success": False, "raw_response": "Connection Error: Ollama server not reachable."}
    except requests.exceptions.Timeout:
        logger.error(f"Ollama request timed out for SRS generation with model {model_name}.")
        return {"success": False, "raw_response": "Timeout Error: Ollama request took too long."}
    except requests.exceptions.HTTPError as http_err:
        logger.error(f"HTTP error occurred: {http_err} - {response.text} with model {model_name}")
        return {"success": False, "raw_response": f"HTTP Error: {http_err}"}
    except Exception as e:
        logger.error(f"An unexpected error occurred during Ollama call: {e} with model {model_name}")
        return {"success": False, "raw_response": f"Unexpected Error: {e}"}

# --- 6. Functions for each Prompt Strategy ---

def run_srs_generation(prompt_name: str, prompt_content: str):
    """
    Runs SRS generation for all LLMs using a specific prompt strategy.
    """
    print(f"\n\n{'#'*10} STARTING EXPERIMENTS FOR PROMPT: {prompt_name} {'#'*10}")
    
    # Print the exact input data for this run for verification
    print("\n--- EXACT CLASSIFIED FEEDBACK USED FOR THIS RUN ---")
    print(CLASSIFIED_FEEDBACK_INPUT_GLOBAL)
    print("-" * 40)

    current_prompt_outputs = {}
    for current_model_name in OLLAMA_MODELS_TO_TEST:
        print(f"\n{'='*20} Generating SRS with Model: {current_model_name} for Prompt: {prompt_name} {'='*20}")
        start_time = time.time()
        
        srs_response = generate_srs_with_ollama(CLASSIFIED_FEEDBACK_INPUT_GLOBAL, current_model_name, prompt_content)
        
        elapsed = time.time() - start_time
        
        if srs_response["success"]:
            print(f"\n--- Generated SRS for {current_model_name} (took {elapsed:.2f} seconds) ---")
            print(srs_response["raw_response"])
            current_prompt_outputs[current_model_name] = srs_response["raw_response"]
        else:
            print(f"\n--- SRS Generation FAILED for {current_model_name}: {srs_response['raw_response']} ---")
            current_prompt_outputs[current_model_name] = f"Failed: {srs_response['raw_response']}"
        
        print(f"\n{'='*20} SRS Generation for {current_model_name} with {prompt_name} Complete {'='*20}\n")
    
    print(f"\n--- Summary for Prompt Strategy: {prompt_name} ---")
    for model, output in current_prompt_outputs.items():
        if output.startswith("Failed"):
            print(f"  {model}: {output}")
        else:
            print(f"  {model}: Successfully generated SRS (Length: {len(output)} characters)")
    print(f"{'#'*10} END OF EXPERIMENTS FOR PROMPT: {prompt_name} {'#'*10}\n")
    return current_prompt_outputs # Return the outputs for easy collection if needed


def run_base_prompt_strategy():
    return run_srs_generation("Base Structured Instructional Prompt", BASE_PROMPT)

def run_role_based_prompt_strategy():
    return run_srs_generation("Role-Based Prompting", ROLE_BASED_PROMPT)

def run_constraint_based_prompt_strategy():
    return run_srs_generation("Constraint-Based Prompting", CONSTRAINT_BASED_PROMPT)

def run_few_shot_prompt_strategy():
    return run_srs_generation("Few-Shot Prompting", FEW_SHOT_PROMPT)

def run_cot_prompt_strategy():
    return run_srs_generation("Chain-of-Thought (CoT) Inspired Prompting", COT_PROMPT)


# --- How to use in Jupyter Cells ---
# Cell 1 (Run this cell once to initialize data and functions):
# [Paste all code above this line]

# Cell 2 (To view the exact data sample used for all runs):
# print(CLASSIFIED_FEEDBACK_INPUT_GLOBAL)

# Cell 3 (To run Base Prompt):
# base_prompt_outputs = run_base_prompt_strategy()
# print(base_prompt_outputs) # Optional: to see the collected outputs

# Cell 4 (To run Role-Based Prompt):
# role_based_outputs = run_role_based_prompt_strategy()
# print(role_based_outputs)

# And so on for other prompt strategies...

--- Initializing Data Loading and Fixed Sample Preparation ---
Global Fixed Sample Prepared: 1278 reviews (NFRs: 1278).
This exact sample will be used for all prompt strategies.
----------------------------------------


In [2]:
base_prompt_outputs = run_base_prompt_strategy()
print(base_prompt_outputs) 



########## STARTING EXPERIMENTS FOR PROMPT: Base Structured Instructional Prompt ##########

--- EXACT CLASSIFIED FEEDBACK USED FOR THIS RUN ---
- Performance: speedy connect
- Usability: why does everything has to be on a white screen
- Portability: the ios 8.3 for my ipad mini 2 screwed up my ibooks bookcase
- Reliability: i've been using the kindle app ever since i ran into the bug
- Reliability: listen up bubs, you'd best [exletive] interface this [exletive] app like the [exletive] app store because i will rage if this thing doesn't load properly
- Performance: it won't work smoothly when i slide the screen for my contact list ,chats and call log
- Reliability: the app closes itself about 5 seconds after opening every single time
- Usability: minimalism as to the design is fine 
- Reliability: when i went back to the app store to get it, my only option was to  open  ibooks but the button didn't do anything
- Reliability: one thing that video quality of the app is very bad
- Relia

In [3]:
role_based_outputs = run_role_based_prompt_strategy()
#print(base_prompt_outputs)



########## STARTING EXPERIMENTS FOR PROMPT: Role-Based Prompting ##########

--- EXACT CLASSIFIED FEEDBACK USED FOR THIS RUN ---
- Performance: speedy connect
- Usability: why does everything has to be on a white screen
- Portability: the ios 8.3 for my ipad mini 2 screwed up my ibooks bookcase
- Reliability: i've been using the kindle app ever since i ran into the bug
- Reliability: listen up bubs, you'd best [exletive] interface this [exletive] app like the [exletive] app store because i will rage if this thing doesn't load properly
- Performance: it won't work smoothly when i slide the screen for my contact list ,chats and call log
- Reliability: the app closes itself about 5 seconds after opening every single time
- Usability: minimalism as to the design is fine 
- Reliability: when i went back to the app store to get it, my only option was to  open  ibooks but the button didn't do anything
- Reliability: one thing that video quality of the app is very bad
- Reliability: however 

In [4]:
constraint_based_outputs = run_constraint_based_prompt_strategy()



########## STARTING EXPERIMENTS FOR PROMPT: Constraint-Based Prompting ##########

--- EXACT CLASSIFIED FEEDBACK USED FOR THIS RUN ---
- Performance: speedy connect
- Usability: why does everything has to be on a white screen
- Portability: the ios 8.3 for my ipad mini 2 screwed up my ibooks bookcase
- Reliability: i've been using the kindle app ever since i ran into the bug
- Reliability: listen up bubs, you'd best [exletive] interface this [exletive] app like the [exletive] app store because i will rage if this thing doesn't load properly
- Performance: it won't work smoothly when i slide the screen for my contact list ,chats and call log
- Reliability: the app closes itself about 5 seconds after opening every single time
- Usability: minimalism as to the design is fine 
- Reliability: when i went back to the app store to get it, my only option was to  open  ibooks but the button didn't do anything
- Reliability: one thing that video quality of the app is very bad
- Reliability: ho

In [5]:
few_shot_outputs = run_few_shot_prompt_strategy()



########## STARTING EXPERIMENTS FOR PROMPT: Few-Shot Prompting ##########

--- EXACT CLASSIFIED FEEDBACK USED FOR THIS RUN ---
- Performance: speedy connect
- Usability: why does everything has to be on a white screen
- Portability: the ios 8.3 for my ipad mini 2 screwed up my ibooks bookcase
- Reliability: i've been using the kindle app ever since i ran into the bug
- Reliability: listen up bubs, you'd best [exletive] interface this [exletive] app like the [exletive] app store because i will rage if this thing doesn't load properly
- Performance: it won't work smoothly when i slide the screen for my contact list ,chats and call log
- Reliability: the app closes itself about 5 seconds after opening every single time
- Usability: minimalism as to the design is fine 
- Reliability: when i went back to the app store to get it, my only option was to  open  ibooks but the button didn't do anything
- Reliability: one thing that video quality of the app is very bad
- Reliability: however i 

In [6]:
cot_outputs = run_cot_prompt_strategy()



########## STARTING EXPERIMENTS FOR PROMPT: Chain-of-Thought (CoT) Inspired Prompting ##########

--- EXACT CLASSIFIED FEEDBACK USED FOR THIS RUN ---
- Performance: speedy connect
- Usability: why does everything has to be on a white screen
- Portability: the ios 8.3 for my ipad mini 2 screwed up my ibooks bookcase
- Reliability: i've been using the kindle app ever since i ran into the bug
- Reliability: listen up bubs, you'd best [exletive] interface this [exletive] app like the [exletive] app store because i will rage if this thing doesn't load properly
- Performance: it won't work smoothly when i slide the screen for my contact list ,chats and call log
- Reliability: the app closes itself about 5 seconds after opening every single time
- Usability: minimalism as to the design is fine 
- Reliability: when i went back to the app store to get it, my only option was to  open  ibooks but the button didn't do anything
- Reliability: one thing that video quality of the app is very bad
- 